# Performing z-m-M cosmology fitting

In [ ]:
import cosmographi as cg
import caskade as ck
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
from time import time

print(jax.config.jax_enable_x64)

## Making a SN sample

In [ ]:
# Define the simulator
z_max = 2.0
cosmology = cg.Cosmology()
rateIa = cg.sn.SNRateConst(cosmology, 1e-4, z_min=0.0, z_max=z_max, name="rateIa")
rateIbc = cg.sn.SNRateConst(cosmology, 5e-5, z_min=0.0, z_max=z_max, name="rateIbc")
ratecombined = cg.sn.CombinedSNRate((rateIa, rateIbc), z_min=0.0, z_max=z_max, cosmology=cosmology)
absmagIa = cg.sn.SNAbsMagGaussian(-19.3, 0.01**2, name="absmagIa")
absmagIbc = cg.sn.SNAbsMagGaussian(-17.0, 0.5**2, name="absmagIbc")
detect_Ia = cg.sn.mSigmoidDetect(threshold=-4, scale=0.1, name="detectIa")
detect_Ibc = cg.sn.mSigmoidDetect(threshold=-4.1, scale=0.13, name="detectIbc")

# Define sampling parameters
n_samples = 100
var_m_Ia = lambda z: 0.01**2 * z
var_z_Ia = lambda z: 0.05**2 * z
cov_Ia = lambda key, var_z, var_m: jnp.zeros(
    var_m.shape
)  # jax.random.uniform(key, var_m.shape, minval=0, maxval=jnp.sqrt(var_m * var_z) * 0.7)
var_m_Ibc = lambda z: 1.0**2 * z
var_z_Ibc = lambda z: 0.2**2 * z
cov_Ibc = lambda key, var_z, var_m: jnp.zeros(
    var_m.shape
)  # jax.random.uniform(key, var_m.shape, minval=0, maxval=jnp.sqrt(var_m * var_z) * 0.7)

# Sample SN
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)
# sample SN redshifts
z = ratecombined.sample(subkey, n_samples)
# sample SN types
t = []
for i in range(n_samples):
    key, subkey = jax.random.split(key)
    t.append(ratecombined.sample_type(subkey, z[i]))
t = jnp.array(t)
# Compute true absolute magnitudes
key, subkey1 = jax.random.split(key)
key, subkey2 = jax.random.split(key)
M = jnp.where(t == 0, absmagIa.sample(subkey1, z.shape), absmagIbc.sample(subkey2, z.shape))
# compute true distance modulus
DL = jax.vmap(cosmology.luminosity_distance)(z)
mu = 5 * jnp.log10(DL) - 5
# determine true apparent magnitudes
m = M + mu
# determine observational uncertainties
var_z = jnp.stack((var_z_Ia(z), var_z_Ibc(z)), axis=-1)  # (N, t)
var_m = jnp.stack((var_m_Ia(z), var_m_Ibc(z)), axis=-1)  # (N, t)
key, subkey1 = jax.random.split(key)
key, subkey2 = jax.random.split(key)
cov = jnp.stack(
    (cov_Ia(subkey1, var_m[:, 0], var_z[:, 0]), cov_Ibc(subkey2, var_m[:, 1], var_z[:, 1])),
    axis=-1,
)  # (N, t)
# sample observed apparent magnitudes
key, subkey = jax.random.split(key)
samp_varz = jnp.where(t == 0, var_z[:, 0], var_z[:, 1])
samp_varm = jnp.where(t == 0, var_m[:, 0], var_m[:, 1])
samp_cov = jnp.where(t == 0, cov[:, 0], cov[:, 1])
obs = jax.random.multivariate_normal(
    key,
    mean=jnp.stack((z, m), axis=-1),
    cov=jnp.stack(
        (jnp.stack((samp_varz, samp_cov), axis=-1), jnp.stack((samp_cov, samp_varm), axis=-1)),
        axis=-1,
    ),
)
z_obs = obs[:, 0]
m_obs = obs[:, 1]
obs_mean = obs  # (N, 2)
obs_cov = jnp.stack(
    (
        jnp.stack((var_z, cov), axis=-1),
        jnp.stack((cov, var_m), axis=-1),
    ),
    axis=-1,
)  # (N, t, 2, 2)
# sample detections
key, subkey1 = jax.random.split(key)
key, subkey2 = jax.random.split(key)
detected = jnp.where(
    t == 0, detect_Ia.sample(subkey1, z, m_obs), detect_Ibc.sample(subkey2, z, m_obs)
)

In [ ]:
print(f"Number of detected SNe: {jnp.sum(detected)} / {n_samples}")
plt.scatter(
    z_obs[~detected],
    m_obs[~detected] - M[~detected],
    c=plt.get_cmap("Greys")(0.5 * t[~detected] + 0.3),
    alpha=0.5,
    label="Not detected SNe",
    linewidth=0,
)
plt.scatter(
    z_obs[detected],
    m_obs[detected] - M[detected],
    c=t[detected],
    cmap="viridis",
    alpha=0.5,
    label="Observed SNe",
    linewidth=0,
)
z_line = jnp.linspace(0.1, z_max, 1000)
mu_base = 5 * jnp.log10(jax.vmap(cosmology.luminosity_distance)(z_line)) - 5
plt.plot(
    z_line,
    mu_base,
    "r--",
    label="True Cosmology",
    linewidth=3,
)
cosmology.Omega_m.to_dynamic()
mu_omegam = (
    5 * jnp.log10(jax.vmap(cosmology.luminosity_distance, in_axes=(0, None))(z_line, [0.35])) - 5
)
plt.plot(
    z_line,
    mu_omegam,
    "g--",
    label="Omega_m=0.35 Cosmology",
    linewidth=3,
)
cosmology.Omega_m.to_static()
cosmology.w0.to_dynamic()
mu_w0 = (
    5 * jnp.log10(jax.vmap(cosmology.luminosity_distance, in_axes=(0, None))(z_line, [-0.9])) - 5
)
plt.plot(
    z_line,
    mu_w0,
    "b--",
    label="w0=-0.9 Cosmology",
    linewidth=3,
)
cosmology.w0.to_static()
plt.xlabel("Redshift (z_obs)")
plt.xscale("log")
plt.ylabel("Observed Distance Modulus (m_obs - M_true)")
plt.legend()
plt.show()

plt.plot(z_line, mu_base - mu_omegam, "g--", label="Omega_m=0.4 Cosmology", linewidth=3)
plt.plot(z_line, mu_base - mu_w0, "b--", label="w0=0.0 Cosmology", linewidth=3)
plt.axhline(0, color="r", linestyle="--", label="True Cosmology", linewidth=3)
plt.xlabel("Redshift (z_obs)")
plt.xscale("log")
plt.ylabel("Distance Modulus Difference")
plt.legend()
plt.show()

## Construct posterior density

In [ ]:
cosmo_mean = jnp.array([0.307, -1])  # Omega_m, w
cosmo_cov = jnp.array([[0.1, 0], [0, 1.0]])
cosmology.Omega_m.to_dynamic()
cosmology.w0.to_dynamic()

detect_Ia_mean = jnp.array([-4, 0.1])  # threshold, scale
detect_Ia_cov = jnp.array([[1.0, 0], [0, 0.01]])
# detect_Ia.threshold.to_dynamic()
# detect_Ia.scale.to_dynamic()

detect_Ibc_mean = jnp.array([-4.1, 0.1])  # threshold, scale
detect_Ibc_cov = jnp.array([[1.0, 0], [0, 0.01]])
# detect_Ibc.threshold.to_dynamic()
# detect_Ibc.scale.to_dynamic()

In [ ]:
class Likelihood(ck.Module):
    def __init__(
        self,
        cosmology: cg.Cosmology,
        ratecombined: cg.sn.CombinedSNRate,
        detect: list[cg.sn.mSigmoidDetect],
        absmag: list[cg.sn.SNAbsMagGaussian],
        name=None,
    ):
        super().__init__(name=name)
        self.cosmology = cosmology
        self.rate = ratecombined
        self.detect = detect
        self.absmag = absmag
        self.vsll = jax.vmap(
            lambda _xy, _i, _z, _t, _M: self.logP_m_theta(_i, _xy, z=_z, t=_t, M=_M),
            in_axes=(0, None, None, None, None),
        )

    @ck.forward
    def Z_zm_theta(self, i, z, t, M):
        mu = 5 * jnp.log10(self.cosmology.luminosity_distance(z)) - 5
        sigma = jnp.sqrt(var_m[i][t])
        m_obs_ = cg.utils.midpoints(M + mu - 5 * sigma, M + mu + 5 * sigma, 50)
        dA = m_obs_[1] - m_obs_[0]
        return jax.nn.logsumexp(self.vsll(m_obs_, i, z, t, M)) + jnp.log(dA)

    @ck.forward
    def logP_m_theta(self, i, m_obs_, z, t, M):
        mu = 5 * jnp.log10(self.cosmology.luminosity_distance(z)) - 5
        # P(z_obs, m_obs | z, M, t, Omega)
        po = -0.5 * (
            jnp.log(obs_cov[i][t][1, 1]) + ((m_obs_ - (M + mu)) ** 2) / obs_cov[i][t][1, 1]
        )
        # P(d=1 | m_obs, z, t)
        pd = self.detect[t].log_prob(z, m_obs_)
        return po + pd

    @ck.forward
    def logP_zm_theta(self, i, z, t, M):
        mu = 5 * jnp.log10(self.cosmology.luminosity_distance(z)) - 5
        zm = jnp.array((z, M + mu))
        # P(z_obs, m_obs | z, M, t, Omega)
        r = obs_mean[i] - zm
        po = -0.5 * (
            jnp.linalg.slogdet(obs_cov[i][t])[1] + jnp.dot(r, jnp.linalg.solve(obs_cov[i][t], r))
        )
        # P(d=1 | m_obs, z, t)
        pd = self.detect[t].log_prob(z, obs_mean[i][1])
        return po + pd

    @ck.forward
    def _log_likelihood_M(self, i, z, t, M):
        ll = self.absmag[t].logP_M(M)
        # P(m_obs | Omega, z, M, d=1, t)
        ll = ll + self.logP_zm_theta(i, z, t, M) - self.Z_zm_theta(i, z, t, M)
        return ll

    @ck.forward
    def _log_likelihood_t(self, i, z, t):
        # int M
        ll = self.rate.logPt_z(t, z)
        M = cg.utils.midpoints(
            self.absmag[t].mean_M.value - 5 * jnp.sqrt(self.absmag[t].var_M.value),
            self.absmag[t].mean_M.value + 5 * jnp.sqrt(self.absmag[t].var_M.value),
            50,
        )
        ll = ll + jax.vmap(self._log_likelihood_M, in_axes=(None, None, None, 0))(i, z, t, M)
        return jax.nn.logsumexp(ll) + jnp.log(M[1] - M[0])
        pass

    @ck.forward
    def _log_likelihood_z(self, i, z):
        # int t
        ll = self.rate.logPz(z)
        log_like = []
        for t in range(len(self.detect)):
            log_like.append(self._log_likelihood_t(i, z, t))
        return jax.nn.logsumexp(jnp.array(log_like)) + ll

    @ck.forward
    def _log_likelihood(self, i):
        # int z
        z = cg.utils.midpoints(
            z_obs[i] - 5 * jnp.sqrt(var_z[i][0]), z_obs[i] + 5 * jnp.sqrt(var_z[i][0]), 50
        )
        ll = jax.vmap(self._log_likelihood_z, in_axes=(None, 0))(i, z)
        return jax.nn.logsumexp(ll) + jnp.log(z[1] - z[0])

    @ck.forward
    def log_likelihood(self):
        return jax.vmap(self._log_likelihood)(jnp.arange(len(m_obs))).sum()


class Prior(ck.Module):
    def __init__(self, cosmology, detect, name=None):
        super().__init__(name=name)
        self.cosmology = cosmology
        self.detect = detect

    @ck.forward
    def log_prior(self):
        prior = 0
        # Cosmology prior
        params = jnp.array([self.cosmology.Omega_m.value, self.cosmology.w0.value])
        # diff = params - jnp.array(cosmo_mean)
        # exponent = -0.5 * jnp.dot(diff, jnp.linalg.solve(cosmo_cov, diff))
        # norm = -0.5 * (len(params) * jnp.log(2 * jnp.pi) + jnp.linalg.slogdet(cosmo_cov)[1])
        # prior = prior + norm + exponent
        prior = jnp.where(
            (params[0] <= 0) | (params[0] >= 1), -jnp.inf, prior
        )  # exclude unphysical Omega_m

        # Detect_Ia prior
        params = jnp.array([self.detect[0].threshold.value, self.detect[0].scale.value])
        diff = params - jnp.array(detect_Ia_mean)
        exponent = -0.5 * jnp.dot(diff, jnp.linalg.solve(detect_Ia_cov, diff))
        norm = -0.5 * (len(params) * jnp.log(2 * jnp.pi) + jnp.linalg.slogdet(detect_Ia_cov)[1])
        prior = prior + norm + exponent

        # Detect_Ibc prior
        params = jnp.array([self.detect[1].threshold.value, self.detect[1].scale.value])
        diff = params - jnp.array(detect_Ibc_mean)
        exponent = -0.5 * jnp.dot(diff, jnp.linalg.solve(detect_Ibc_cov, diff))
        norm = -0.5 * (len(params) * jnp.log(2 * jnp.pi) + jnp.linalg.slogdet(detect_Ibc_cov)[1])
        prior = prior + norm + exponent
        return prior


class Posterior(ck.Module):
    def __init__(self, likelihood, prior, name=None):
        super().__init__(name=name)
        self.likelihood = likelihood
        self.prior = prior

    @ck.forward
    def log_posterior(self):
        log_prior = self.prior.log_prior()
        log_likelihood = self.likelihood.log_likelihood()
        return log_prior + log_likelihood

In [ ]:
posterior = Posterior(
    likelihood=Likelihood(
        cosmology,
        ratecombined,
        [detect_Ia, detect_Ibc],
        [absmagIa, absmagIbc],
        name="likelihood",
    ),
    prior=Prior(cosmology, [detect_Ia, detect_Ibc]),
)

start = time()
print(posterior.log_posterior())
print("Time taken (s):", time() - start)
start = time()
print(posterior.log_posterior())
print("Time taken (s):", time() - start)
start = time()
print(posterior.log_posterior())
print("Time taken (s):", time() - start)
start = time()
print(posterior.log_posterior())
print("Time taken (s):", time() - start)

print(posterior)

In [ ]:
N = 50
Om = jnp.linspace(0.0, 0.7, N)
Ow = jnp.linspace(-2, 0.0, N)
x = jnp.stack(jnp.meshgrid(Om, Ow, indexing="xy"), axis=-1)
Lp = cg.utils.vmap_chunked1d(posterior.log_posterior, chunk_size=25)(x.reshape(-1, 2)).reshape(N, N)
plt.contourf(Om, Ow, Lp, levels=20, cmap="viridis", vmin=jnp.max(Lp) - 100)
plt.axvline(cosmo_mean[0], color="r", linestyle="--", label="True Omega_m")
plt.axhline(cosmo_mean[1], color="r", linestyle="--", label="True w0")
plt.colorbar(label="Log Posterior")
plt.xlabel("Omega_m")
plt.ylabel("w0")
plt.show()

In [ ]:
plt.contourf(Om, Ow, jnp.clip(Lp, a_min=jnp.max(Lp) - 100), levels=16, cmap="viridis")
plt.axvline(cosmo_mean[0], color="r", linestyle="--", label="True Omega_m")
plt.axhline(cosmo_mean[1], color="r", linestyle="--", label="True w0")
plt.colorbar(label="Log Posterior")
plt.xlabel("Omega_m")
plt.ylabel("w0")
plt.show()

In [ ]:
plt.imshow(
    Lp,
    cmap="viridis",
    vmin=jnp.max(Lp) - 100,
    origin="lower",
    extent=(0, 0.7, -2, 0),
    interpolation="none",
)
plt.gca().set_aspect((0.7 - 0) / (0 - -2))
plt.axvline(cosmo_mean[0], color="r", linestyle="--", label="True Omega_m")
plt.axhline(cosmo_mean[1], color="r", linestyle="--", label="True w0")
plt.colorbar(label="Log Posterior")
plt.xlabel("Omega_m")
plt.ylabel("w0")
plt.show()

In [ ]:
posterior.likelihood.z.to_dynamic()
posterior.likelihood.M.to_dynamic()
x0 = posterior.build_params_array()
print(x0.shape)

chains = 16
key, subkey = jax.random.split(key)

# warmup
key, subkey = jax.random.split(key)
samples_burn = cg.utils.mala(
    jnp.repeat(x0[None, :], chains, axis=0),
    posterior.log_posterior,
    num_samples=1000,
    epsilon=0.001,
    key=subkey,
    progress=True,
)
cov = jnp.cov(samples_burn[int(samples_burn.shape[0] / 2) :].reshape(-1, samples_burn.shape[-1]).T)
print(cov.shape)

plt.plot(range(samples_burn.shape[0]), samples_burn[:, 0, 0], alpha=0.5, label="Omega_m, chain 0")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 1, 0], alpha=0.5, label="Omega_m, chain 1")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 2, 0], alpha=0.5, label="Omega_m, chain 2")
plt.xlabel("Sample")
plt.ylabel("Omega_m")
plt.title("Warmup")
plt.legend()
plt.show()

plt.plot(range(samples_burn.shape[0]), samples_burn[:, 0, 1], alpha=0.5, label="w0, chain 0")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 1, 1], alpha=0.5, label="w0, chain 1")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 2, 1], alpha=0.5, label="w0, chain 2")
plt.xlabel("Sample")
plt.ylabel("w0")
plt.title("Warmup")
plt.legend()
plt.show()

plt.plot(range(samples_burn.shape[0]), samples_burn[:, 0, 2], alpha=0.5, label="z, chain 0")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 1, 2], alpha=0.5, label="z, chain 1")
plt.plot(range(samples_burn.shape[0]), samples_burn[:, 2, 2], alpha=0.5, label="z, chain 2")
plt.xlabel("Sample")
plt.ylabel("z")
plt.title("Warmup")
plt.legend()
plt.show()

# production
key, subkey = jax.random.split(key)
samples_prod = cg.utils.mala(
    samples_burn[-1],
    posterior.log_posterior,
    mass_matrix=cov,
    num_samples=10000,
    epsilon=0.1,
    key=subkey,
    progress=True,
)

In [ ]:
plt.plot(range(samples_prod.shape[0]), samples_prod[:, 0, 0], alpha=0.5, label="Omega_m, chain 0")
plt.plot(range(samples_prod.shape[0]), samples_prod[:, 1, 0], alpha=0.5, label="Omega_m, chain 1")
plt.plot(range(samples_prod.shape[0]), samples_prod[:, 2, 0], alpha=0.5, label="Omega_m, chain 2")
plt.xlabel("Sample")
plt.ylabel("Omega_m")
plt.title("Production")
plt.legend()
plt.show()

In [ ]:
print(posterior.dynamic_params)
print(posterior)

In [ ]:
plt.hist(samples_prod[:, :, 0].reshape(-1), bins=30, density=True)
plt.axvline(x0[0], color="r", linestyle="--", label="True Omega_m")
plt.xlabel("Omega_m")
plt.legend()
plt.show()

In [ ]:
plt.hist(samples_prod[:, :, 1].reshape(-1), bins=30, density=True)
plt.axvline(x0[1], color="r", linestyle="--", label="True w0")
plt.xlabel("w0")
plt.legend()
plt.show()

In [ ]:
plt.scatter(samples_prod[:, :, 0].reshape(-1), samples_prod[:, :, 1].reshape(-1), s=0.1, alpha=0.1)
plt.axvline(x0[0], color="r", linestyle="--", label="True Omega_m")
plt.axhline(x0[1], color="r", linestyle="--", label="True w0")
plt.xlabel("Omega_m")
plt.ylabel("w0")
plt.legend()
plt.show()

In [ ]:
for i in range(5):
    plt.hist(samples_prod[:, :, 2 + i].reshape(-1), bins=30, density=True)
    plt.axvline(x0[2 + i], color="r", linestyle="--", label=f"True z[{i}]")
    plt.xlabel(f"z[{i}]")
    plt.legend()
    plt.show()

In [ ]:
len(x0)